!pip install simpleitk
!pip install numpy
!pip install pynrrd
!pip install opencv-python

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os as os
import nrrd as reader
import pandas as pd
import cv2 
import csv
from tqdm import tqdm
from tabulate import tabulate
from skimage import morphology
from skimage import measure
from skimage import segmentation
from skimage.segmentation import flood, flood_fill
from skimage.measure import label
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters
from time import sleep
from IPython.display import clear_output

In [ ]:
def PatientViewer (Array, ArrayN, Array1, ArrayN1, Array2, ArrayN2):
    s=1
    for i in range (0,totalslices,slicesperpatient):
        fig, axs=plt.subplots (1,3, figsize=(15,10))
        axs[0].imshow(Array[i],"bone",vmin=0,vmax=256)
        axs[0].set_title(f'{ArrayN} Patient {s}')
        axs[0].axis("off")
        axs[1].imshow(Array1[i],"bone",vmax=256)
        axs[1].set_title(f'{ArrayN1} Patient {s}')
        axs[1].axis("off")
        axs[2].imshow(Array2[i],"bone",vmin=0,vmax=255)
        axs[2].set_title(f'{ArrayN2} Patient {s}')
        axs[2].axis("off")
        s+=1

In [ ]:
def join(one, two):
    return os.path.join(one, two)

In [ ]:
def check_path(path):
    count=0
    for files in sorted(os.listdir(path)):
        image_path = os.path.join(path,files)
        count+=1
    return count

In [ ]:
#functions for inhomogeneity correction 
def correct_roi(image):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    return image_c

In [ ]:
def dcm_to_np(path):
    global dic_count
    try:
        slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path)
        image = sitk.ReadImage(slice_filenames)
        for x in range(slicesperpatient):
            current_slice = image[:, :, x]
            current_array = sitk.GetArrayFromImage(current_slice)
            split_array = current_array[:,:-256]
            inputs[dic_count] = split_array[:, :, np.newaxis]
            dic_count+=1
        print(f'{patient} {R}/{num_patients}')
    except Exception as e:
        print(e)

In [ ]:
def nrrd_to_np(meta_object,outputs):
    global ann_count
    try:
        data = sitk.GetArrayFromImage(meta_object)
        for p in range(slicesperpatient):
            current_seg = data[p, :, :]
            half_seg= current_seg[:,:-256]
            black = np.zeros([256, 256, 9])
            for y, a in enumerate(half_seg):
                for x, b in enumerate(a):
                    if b>0:
                        black[y][x][b-1] = 1
            outputs[ann_count] = black
            ann_count+=1
        print("Segmentation Imported")
    except Exception as e:
        print(e)

In [ ]:
def fle_to_np(meta_object,outputs):
    global fle_count
    try:
        data = sitk.GetArrayFromImage(meta_object)
        for p in range(slicesperpatient):
            outputs[fle_count] = np.expand_dims(np.where(data[p, :, :256] == 1, 1, 0), axis=2)
            fle_count+=1
        print('FLE IMPORTED')        
    except Exception as e:
        print(e)

In [ ]:
def npnormalization (inputimage):
    for x in range (5): #NumPy Normalization  
        clear_output(wait=True)
        temp = inputimage[x].reshape(256, 256)
        temp1= correct_roi(temp)
        np.linalg.norm(temp1)
        inputs[x]=temp1[:, :, np.newaxis]
        print(f'{x+1}/{totalslices} Slices |{round((x/totalslices)*100)}%|')
    print(f'Normalization Complete')
    return inputs

In [ ]:
def imageprocessor(Array,Image,Segmentations,FLESegmentations):
    print("Processing DICOMs with Imported Segmentations... Please Wait...")
    imageI = Array[:, :, :, 0] #Inputs Variable 
    ArrayI = np.empty((num_patients * Image.GetSize()[2], int(Image.GetSize()[0]/2), int(Image.GetSize()[1]), 1))
    mskROI = np.empty((num_patients * Image.GetSize()[2], int(Image.GetSize()[0]/2), int(Image.GetSize()[1]), 1))
    fleoutputsI = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
    boneremoval = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
    for x in range (totalslices): 
        for i in range (9):
            if i !=2:
                ArrayI[x,:,:,0]+=Segmentations[x,:,:,i] #Merged MSK Mask
    FArrayI = FLESegmentations[:,:,:,0]
    boneremoval = Segmentations[:,:,:,2]
    ArrayII = ArrayI[:, :, :, 0] # Get rid of [:,:,:,1]
    INVArrayI = 1 - FArrayI #Invered Mask
    INVArrayII = INVArrayI * imageI #Mask multipled with array
    arraymean = np.nanmean(np.where(Array!=0,Array,np.nan))
    arraystd = np.nanstd(np.where(Array!=0,Array,np.nan))
    print(f'Array Mean - 1.5 STD = {arraymean-1.5*arraystd}')
    INVArrayIII = np.where(INVArrayII > (arraymean-1.5*arraystd), 1, 0)
    FArrayII = INVArrayIII + FArrayI #Add both the merged msk and inv msk together
    
    print("Pre Process-------------------------------------------")
    for i in range (0,totalslices,slicesperpatient):
        plt.imshow(imageI[i],cmap=plt.cm.bone,vmin=0,vmax=255)
        plt.show()
        
    FinalArray = FArrayII * imageI #Multiply new mask with original image
    mskROI = imageI * ArrayII
    fleROI = imageI * (FArrayI-boneremoval)
    print("Post Process-------------------------------------------")
    for i in range (0,totalslices,slicesperpatient):
        plt.imshow(FinalArray[i],cmap=plt.cm.bone,vmin=0,vmax=255)
        plt.show()
        
    return FinalArray,mskROI,fleROI,ArrayII
    

In [ ]:
def multi_otsu(image):
    motsuth=filters.threshold_multiotsu(image, classes=3)
    print(motsuth)
    print (f"Slice {i+1} otsu threshold={motsuth[1]}")
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)
    return motsuth[1] #(fat+ muscle th)

In [ ]:
def connectivity(fatmask,OG_VOL,z_connection):
        fatmask = np.uint8(fatmask)
        if i==0:
            fatcombined_next=OG_VOL[i+1]+fatmask
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
        elif i==(totalslices-1):
            fatcombined_prev=OG_VOL[i-1]+fatmask
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev  
        else:
            fatcombined_prev=OG_VOL[i-1]+fatmask
            fatcombined_next=fatmask+OG_VOL[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)         
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1)    
        im_ff=fatmask.copy()
        h, w = im_ff.shape[:2] #added
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2) 
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)
        Z = (im_ff==2)
        prefatseg1=(Z+nonZ_keep) #do we need int cus boolean??
        fatseg1=prefatseg1*OG_VOL[i]
        return fatseg1

In [ ]:
def ITSA(i,InROI, ots, OG_VOL):
    k=1
    ThPrev=0 
    ThRev= ots[i] 
    x=0
    y=0
    
    while ThRev!=ThPrev:
        ThPrev=ThRev
        prefatmask = (InROI[i]>ThRev)
        prefatmask = np.uint8(prefatmask)
        ret, fatmask = cv2.threshold(prefatmask,0,1,cv2.THRESH_BINARY)  #need this??
        if i==0:
            fatcombined_next=OG_VOL[i+1]+fatmask
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
        elif i==(totalslices-1):
            fatcombined_prev=OG_VOL[i-1]+fatmask
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev  
        else:
            fatcombined_prev=OG_VOL[i-1]+fatmask
            fatcombined_next=fatmask+OG_VOL[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)         
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
        im_ff=fatmask.copy()
        h, w = im_ff.shape[:2] #added
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2) 
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)
        Z = (im_ff==2)
        prefatseg1=(Z+nonZ_keep) #do we need int cus boolean??
        fatseg1=prefatseg1*InROI[i]
        #Fat and Muscle Quantification
        preMuscSegP=InROI[i]-fatseg1
        MuscSegP=np.ma.masked_where(preMuscSegP == 0, preMuscSegP)
        MuscSeg = InROI[i]*MuscSegP
        FatSegP=np.ma.masked_where(fatseg1==0,fatseg1) 
        plt.imshow(FatSegP)
        MuscSegI=np.mean(MuscSegP)
        FatSegI=np.mean(FatSegP)
        ThRev=(1+((FatSegI-MuscSegI)/FatSegI))*MuscSegI
        print (f"Slice #{i+1} Iteration={k}\n\tThPrev={ThPrev}\n\tThRev={ThRev}\n")

        k+=1
        if k==50:
            break
    
    
    thresholds = ThRev
    
    return fatseg1,thresholds

In [ ]:
def correction(SegmentedFatROI,ROIArray):
    print('Muscle Segmentation')
    plt.hist(np.where(ROIArray[1] !=0,ROIArray[1],np.nan))
    plt.show()
    print('Fat Segmentation')
    plt.hist(np.where(SegmentedFatROI[1] !=0,SegmentedFatROI[1],np.nan))
    plt.show()
    
    musclemean=np.nanmean(np.where(ROIArray !=0,ROIArray,np.nan))
    musclestd=np.nanstd(np.where(ROIArray !=0,ROIArray,np.nan))
    print("mean")
    print(musclemean)
    print("standard dev")
    print(musclestd)

    MUSCLETHRES = int(musclemean+(2*musclestd))
    print('Muscle Threshold')
    print(MUSCLETHRES)
    fatseg2 = np.where(SegmentedFatROI > MUSCLETHRES, SegmentedFatROI, 0) 
    
    print("before")
    plt.imshow(SegmentedFatROI[1],cmap='bone',vmin=0,vmax=256)
    plt.show()
    print("after")
    plt.imshow(fatseg2[1],cmap='bone',vmin=0,vmax=256)
    plt.show()
    fatseg2
    
    return fatseg2
    

In [ ]:
def dataexport(ROIimage,FatSegmentation,MRImage):
    muscfatarea=np.empty([num_patients,1])
    fatarea=np.empty([num_patients,1])
    muscarea=np.empty([num_patients,1])
    fatperc=np.empty([num_patients,1])
    muscfatvol=np.empty([num_patients,1])
    fatvol=np.empty([num_patients,1])
    muscvol=np.empty([num_patients,1])
    muscleroi=ROIimage-FatSegmentation
    s=0
    for i in range (0,totalslices,slicesperpatient):
        MuscFatAreaPix=0
        FatAreaPix=0
        MuscAreaPix=0
        MuscFatAreaCM=0
        FatAreaCM=0
        MuscAreaCM=0
        for x in range (0,slicesperpatient,1):
            MuscFatAreaPix+=np.sum(ROIimage[i+x]>0)
            FatAreaPix+=np.sum(FatSegmentation[i+x]>0)

        MuscFatAreaCM=MuscFatAreaPix*((MRImage.GetSpacing()[0]*MRImage.GetSpacing()[0])/100) #(InputImage.GetSpacing()[0]*InputImage.GetSpacing()[0])/10#
        FatAreaCM=FatAreaPix*((MRImage.GetSpacing()[0]*MRImage.GetSpacing()[0])/100)

        muscfatarea[s] = MuscFatAreaCM
        fatarea[s] = FatAreaCM
        muscarea[s] = MuscFatAreaCM - FatAreaCM
        
        muscfatvol[s] = muscfatarea[s]*(MRImage.GetSpacing()[2]/10)
        fatvol[s] = fatarea[s]*(MRImage.GetSpacing()[2]/10)
        muscvol[s] = muscarea[s]*(MRImage.GetSpacing()[2]/10)
        fatperc[s] = (fatarea[s]/muscfatarea[s])*100
        print(f'__________________________________________________Patient #{s+1}')
        print(tabulate([[' MUSCLE & FAT AREA', muscfatarea[s], '∑(cm^2)'] , ['FAT AREA', fatarea[s], '∑(cm^2)'],[' MUSCLE AREA', muscarea[s], '∑(cm^2)'],['FAT PERCENTAGE', fatperc[s], '%'],[' MUSCLE & FAT VOLUME', muscfatvol[s], '∑(cm^3)'],['FAT VOLUME',fatvol[s], '∑(cm^3)'],['MUSCLE VOLUME', muscvol[s], '∑(cm^3)']],     headers=['Data', 'Value','Units']))
        s+=1

    PDF=np.empty([num_patients,1])
    PDF=np.append(muscfatarea,fatarea,axis=1)
    PDF=np.append(PDF,muscarea,axis=1)
    PDF=np.append(PDF,fatperc,axis=1)
    PDF=np.append(PDF,muscfatvol,axis=1)
    PDF=np.append(PDF,fatvol,axis=1)
    PDF=np.append(PDF,muscvol,axis=1)
    return PDF

# #CODE SEC --------------------------------------------------

In [ ]:
#       // PATIENT DCM // MSK SEG NRRD // DCM NORMALIZATION // IMG PROCESSOR // FLE PROCESSOR

In [ ]:
num_patients = int(check_path(join(os.getcwd(), "dataset")))
totalslices=(num_patients * InputImage.GetSize()[2])
slicesperpatient=InputImage.GetSize()[2]
folder_path=join(os.getcwd(), "dataset")
for patient in sorted(os.listdir(folder_path)):
    patient_path = join(folder_path, patient)
    InputImage=sitk.ReadImage(sitk.ImageSeriesReader_GetGDCMSeriesFileNames(patient_path)) 
    inputs = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
    sortedpatients= sorted(os.listdir(folder_path))


In [ ]:
mskoutputs = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 9))
fleoutputs=np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
fle_count = 0
dic_count = 0
ann_count = 0
R=1
try:
    for patient in sorted(os.listdir(folder_path)):
        clear_output(wait=True)
        patient_path = join(folder_path, patient)
        nrrd_folder = join(patient_path, "NRRD")
        dcm_to_np(patient_path)
        for nrrd in os.listdir(nrrd_folder):
            if(".nrrd" in nrrd):
                segmentation = sitk.ReadImage(join(nrrd_folder, nrrd))
                nrrd_to_np(segmentation,mskoutputs)
        fle_folder = join(patient_path, "FLE")
        for fle in os.listdir(fle_folder):
            if(".nrrd" in fle):
                flesegmentation = sitk.ReadImage(join(fle_folder, fle))
                fle_to_np(flesegmentation,fleoutputs)
        R+=1
except Exception as e:
    print(e)

In [ ]:
ImportedArray = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
ImportedArray = npnormalization (inputs)

In [ ]:
thigharray=np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1])))
ROI_MSK=np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1])))
ROI_FLE=np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1])))
OG_VOL=np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1])))
thigharray,ROI_MSK,ROI_FLE,OG_VOL=imageprocessor(ImportedArray,InputImage,mskoutputs,fleoutputs)


In [ ]:
fig, axs=plt.subplots (1,3, figsize=(15,10))
axs[0].imshow(mskoutputs[1,:,:,0],"bone",vmin=0,vmax=1)
axs[0].set_title("RF")
axs[0].axis("off")
axs[1].imshow(mskoutputs[1,:,:,1],"bone",vmin=0,vmax=1)
axs[1].set_title('VG')
axs[1].axis("off")
axs[2].imshow(mskoutputs[1,:,:,2],"bone",vmin=0,vmax=1)
axs[2].set_title('Femur')
axs[2].axis("off")

fig, axs=plt.subplots (1,3, figsize=(15,10))
axs[0].imshow(mskoutputs[1,:,:,3],"bone",vmin=0,vmax=1)
axs[0].set_title("Gracilis")
axs[0].axis("off")
axs[1].imshow(mskoutputs[1,:,:,4],"bone",vmin=0,vmax=1)
axs[1].set_title('Sartorius')
axs[1].axis("off")
axs[2].imshow(mskoutputs[1,:,:,5],"bone",vmin=0,vmax=1)
axs[2].set_title('Adductors')
axs[2].axis("off")
    
fig, axs=plt.subplots (1,3, figsize=(15,10))
axs[0].imshow(mskoutputs[1,:,:,6],"bone",vmin=0,vmax=1)
axs[0].set_title("BF")
axs[0].axis("off")
axs[1].imshow(mskoutputs[1,:,:,7],"bone",vmin=0,vmax=1)
axs[1].set_title('ST')
axs[1].axis("off")
axs[2].imshow(mskoutputs[1,:,:,8],"bone",vmin=0,vmax=1)
axs[2].set_title('SM')
axs[2].axis("off")


In [ ]:
plt.imshow(thigharray[1],"bone",vmin=0,vmax=255)
plt.imshow(fleoutputs[1],"reds",vmin=0,vmax=1)

In [ ]:
CorrectedFascia = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
CorrectedFascia = npnormalization (ROI_FLE)
CorrectedFascia=CorrectedFascia[:,:,:,0]

In [ ]:
iThreshold=[]
print('Multi Otsu Threshold...')
for i in range(totalslices):
    iThreshold.append([])
    iThreshold[i]=multi_otsu(CorrectedFascia[i])
print ("-----------------------------------------------------------")


In [ ]:
#ITSA Threshold Seeking Round 1 
fleROI_R1=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8')
fatseg=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8')
fatsegMASK=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8') #change name
threshold_R1=[] #change name compared to single slice?
print("ITSA Round 1 Initiation...")
for i in range(totalslices):
    threshold_R1.append([])
    fatseg[i],threshold_R1[i]=ITSA(i, CorrectedFascia, iThreshold,OG_VOL )  #fatsegfinalvol_mask_R1=round 1 final fat mask with islands REMOVED
    print (f"Slice #{i+1} Round 1 Th1 = {threshold_R1[i]}")
    print ("-----------------------------------------------------------")
    
for i in range(totalslices):
    print (f"Slice #{i+1} Th1 OG/Rev={threshold_R1[i]}")

In [ ]:
fleROI_R1=CorrectedFascia-fatseg
PatientViewer(CorrectedFascia[i],"Original Array",fatseg[i],"Fat Segmentation Round 1",fleROI_R1[i],"Muscle Segmentation Round 1")

In [ ]:
#ITSA Threshold Seeking Round 2
print('Multi Otsu Threshold Round 2...')
iThresholdR2=[]
for i in range(totalslices):
    iThresholdR2.append([])
    iThresholdR2[i]=multi_otsu(fleROI_R1[i])

In [ ]:
fatsegR2=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8') 
threshold_R2=[] 
for i in range(totalslices):
    threshold_R2.append([])
    fatsegR2[i], threshold_R2[i]=ITSA(i, fleROI_R1, iThresholdR2, OG_VOL)  #fatsegfinalvol_mask_R1=round 1 final fat mask with islands REMOVED
    
    print (f"Slice #{i+1} Round 2 Threshold = {threshold_R2[i]}")
    print ("-----------------------------------------------------------")
for i in range(totalslices):
    print (f"Slice #{i+1} Th2 OG/Rev={threshold_R2[i]}")

In [ ]:
CorrectedFatSegmentation=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8')
print('Correcting Potential Segmentation Errors... Please Wait...')
CorrectedFatSegmentation=correction(fatsegR2,fleROI_R1)
FinalFatSegmentation = fatseg+CorrectedFatSegmentation
FinalMuscleSegmentation = ROI_FLE - FinalFatSegmentation
PatientViewer(thigharray,"Original Array",FinalFatSegmentation,"Fat Segmentation (Final Corrected)",FinalMuscleSegmentation,"Muscle Segmentation (Final Corrected)")

In [ ]:
Post3D=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8')
for i in range (150,165):
    Post3D[i]=connectivity(FinalFatSegmentation,thigharray)

In [ ]:
for i in range (150,165):    
    plt.imshow(Post3D[i],'bone',vmax=1)
    plt.show()

In [ ]:
fatmask.shape

In [ ]:
def connectivity(fatmask,OG_VOL):
        fatmask = np.uint8(fatmask)
        ret, fatmask = cv2.threshold(fatmask,0,1,cv2.THRESH_BINARY)
        if i==0:
            fatcombined_next=OG_VOL[i+1]+fatmask[i]
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
        elif i==(totalslices-1):
            fatcombined_prev=OG_VOL[i-1]+fatmask[i]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev  
        else:
            fatcombined_prev=OG_VOL[i-1]+fatmask[i]
            fatcombined_next=fatmask[i]+OG_VOL[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)         
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1)    
        im_ff=fatmask[i].copy()
        h, w = im_ff.shape[:2] #added
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2) 
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)
        Z = (im_ff==2)
        prefatseg1=(Z+nonZ_keep) #do we need int cus boolean??
        return prefatseg1

In [ ]:
PatientViewer(fatseg,"Fat Segmentation Round 1",CorrectedFatSegmentation,"Fat Segmentation Round 2",FinalFatSegmentation,"Fat Segmentation (Final)")

In [ ]:
"ROUND 1 ITSA DATA"
PDFTHR=np.empty([num_patients,1])
PDFTHR=dataexport(ROI_FLE,fatseg,InputImage)

In [ ]:
"FINAL ROUND 2 ITSA DATA"
PDFTHR2=np.empty([num_patients,1])
PDFTHR2=dataexport(ROI_FLE,FinalFatSegmentation,InputImage)

In [ ]:
"Rectus Femoris"
ROIrf=ROI_FLE*mskoutputs[:,:,:,0]
finalfatsegRF=FinalFatSegmentation*mskoutputs[:,:,:,0]
PDFRF=np.empty([num_patients,1])
PDFRF=dataexport(ROIrf,finalfatsegRF,InputImage)

In [ ]:
"Vastus Group"
ROIvg=ROI_FLE*mskoutputs[:,:,:,1]
finalfatsegVG=FinalFatSegmentation*mskoutputs[:,:,:,1]
PDFVG=np.empty([num_patients,1])
PDFVG=dataexport(ROIvg,finalfatsegVG,InputImage)

In [ ]:
"Gracilis"
ROIgr=ROI_FLE*mskoutputs[:,:,:,3]
finalfatsegGR=FinalFatSegmentation*mskoutputs[:,:,:,3]
PDFGR=np.empty([num_patients,1])
PDFGR=dataexport(ROIgr,finalfatsegGR,InputImage)

In [ ]:
"Sartorius"
ROIsr=ROI_FLE*mskoutputs[:,:,:,4]
finalfatsegSR=FinalFatSegmentation*mskoutputs[:,:,:,4]
PDFSR=np.empty([num_patients,1])
PDFSR=dataexport(ROIsr,finalfatsegSR,InputImage)

In [ ]:
"Adductors"
ROIad=ROI_FLE*mskoutputs[:,:,:,5]
finalfatsegAD=FinalFatSegmentation*mskoutputs[:,:,:,5]
PDFAD=np.empty([num_patients,1])
PDFAD=dataexport(ROIad,finalfatsegAD,InputImage)

In [ ]:
"bicepsfemoris"
ROIbf=ROI_FLE*mskoutputs[:,:,:,6]
finalfatsegBF=FinalFatSegmentation*mskoutputs[:,:,:,6]
PDFBF=np.empty([num_patients,1])
PDFBF=dataexport(ROIbf,finalfatsegBF,InputImage)

In [ ]:
"semitendinosus"
ROIst=ROI_FLE*mskoutputs[:,:,:,7]
finalfatsegST=FinalFatSegmentation*mskoutputs[:,:,:,7]
PDFST=np.empty([num_patients,1])
PDFST=dataexport(ROIst,finalfatsegST,InputImage)

In [ ]:
"Semimembranosus"
ROIsm=ROI_FLE*mskoutputs[:,:,:,8]
finalfatsegSM=FinalFatSegmentation*mskoutputs[:,:,:,8]
PSFSM=np.empty([num_patients,1])
PDFSM=dataexport(ROIsm,finalfatsegSM,InputImage)

In [ ]:
FINALPDF=np.empty([num_patients,0])
FINALPDF=np.append(FINALPDF,PDFTHR,axis=1)
FINALPDF=np.append(FINALPDF,PDFTHR2,axis=1)
FINALPDF=np.append(FINALPDF,PDFRF,axis=1)
FINALPDF=np.append(FINALPDF,PDFVG,axis=1)
FINALPDF=np.append(FINALPDF,PDFGR,axis=1)
FINALPDF=np.append(FINALPDF,PDFSR,axis=1)
FINALPDF=np.append(FINALPDF,PDFAD,axis=1)
FINALPDF=np.append(FINALPDF,PDFBF,axis=1)
FINALPDF=np.append(FINALPDF,PDFST,axis=1)
FINALPDF=np.append(FINALPDF,PDFSM,axis=1)

In [ ]:
msk = ["Thigh R1 ","Thigh R2 (ΣR1&2) ", "Rectus Femoris ", "Vastus Group ", "Gracilis ", "Sartorius ", "Adductors ", "Biceps Femoris ", "Semitendinosus ", "Semimembranosus "]
var = ["MUSCLE & FAT AREA (CM^2)", "FAT AREA (CM^2)","MUSCLE AREA (CM^2)", "FAT PERCENTAGE (%)", "MUSCLE & FAT VOL (CM^3)", "FAT VOL (CM^3)", "MUSCLE VOL (CM^3)" ]
for j in (msk):
    for k in (var):
        lst = [(j + k) for j in msk for k in var]
df = pd.DataFrame(FINALPDF, index=sorted(os.listdir(folder_path)), columns=lst)
df.to_csv('ITSAOAITHIGHDATA.csv')


In [ ]:
OG_VOL.shape

In [ ]:
ImportedArray.shape

In [ ]:
def get_snakeim():
    snake_im=np.zeros([thigharray.shape[0], thigharray.shape[1], thigharray.shape[2]], dtype='uint8')
    snake_im=ImportedArray[:,:,:,0]
    for i in range(5):
        snake_im[i] = np.where(OG_VOL[i]==1,255,snake_im[i]) #where the coordinates on white=1 (vessels), on snake_im make it make those coordaintes it 255 (make the vessel coordinates white to remove them)
    return snake_im

snake_im=get_snakeim()

In [ ]:
plt.imshow(snake_im[1],cmap='bone')

In [ ]:
prefloodfill = np.empty((num_patients * InputImage.GetSize()[2], int(InputImage.GetSize()[0]/2), int(InputImage.GetSize()[1]), 1))
prefloodfill = npnormalization (snake_im)
prefloodfill = prefloodfill[:,:,:,0]


In [ ]:
plt.imshow(prefloodfill[1],cmap='bone')

In [ ]:

def bilat_fil_snake_im(image):
    bilateral_t=np.zeros([inputs.shape[0], inputs.shape[1], inputs.shape[2]], dtype='uint8')
    for i in range(5):
        bilateral_t[i] = cv2.bilateralFilter(image[i],20,35,35)
    return bilateral_t

bilateral_t=bilat_fil_snake_im(np.uint8(snake_im))
plt.imshow(bilateral_t[1],cmap='bone')

In [ ]:
j=5

In [ ]:
contoursvol=[]
hiercvol=[]
contour_listvol=[]
maskoutlinevol=[]

r=np.zeros([inputs.shape[0], inputs.shape[1], inputs.shape[2]], dtype='uint8')

OG_VOL2=cv2.convertTo(OG_VOL,OG_VOL2,CV_BGR2GRAY)
for i in range(j):
    contoursvol.append([])
    hiercvol.append([])
    contour_listvol.append([])
    maskoutlinevol.append([])
    
for i in range (j):
    a, b =  cv2.findContours(OG_VOL[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contoursvol[i].append(a)
x    hiercvol[i].append(b)
    
    for contour in contoursvol[i][0]:      
        contour_listvol[i].append(contour)

    maskoutlinevol[i] = cv2.drawContours(r[i], contour_listvol[i],  -1, (1,0,0), 1)

In [ ]:
# Find contours at a constant value of 0.8
contours = measure.find_contours(ROI_MSK[1,:,:],0)
plt.imshow(contours[90])
plt.show()

In [ ]:
# Find contours at a constant value of 0.8
contours = measure.find_contours(ROI_MSK[1],0)
plt.imshow
# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(inputs[1], cmap=plt.cm.gray)

for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
inpus=np.zeros([inputs.shape[0], inputs.shape[1], inputs.shape[2]], dtype='object')
inpus=np.where(contours,1,0)

In [ ]:
contour_listvol=[]
maskoutlinevol=[]
contoursvol=[]
r=np.zeros([inputs.shape[0], inputs.shape[1], inputs.shape[2]], dtype='uint8')
for i in range(10):
    contoursvol.append([])
    contour_listvol.append([])
    maskoutlinevol.append([])
    
maskoutlinevol[1] = cv2.drawContours(r[1], contours[1],  -1, (1,0,0), 1)

In [ ]:
def fill_contours(arr):
    return np.maximum.accumulate(arr,1) & \
           np.maximum.accumulate(arr[:,::-1],1)[:,::-1]

fill_contours(OG_VOL[1])

In [ ]:
hull2vol=[]

for i in range(5):
    hull2vol.append([])
    for i2 in range(len(OG_VOL[i][0][0])):
        hull2vol[i].append(OG_VOL[i][0][0][i2][0])
        

    hull2vol[i]=np.array(hull2vol[i])
    hull2vol[i]=hull2vol[i].astype(float)

In [ ]:
#primary
def primary_snake():

    snakevol_p=[]
    for i in range(5):
        snakevol_p.append([])
        snakevol_p[i]= active_contour(bilateral_t[i], hull2vol[i],alpha=0.0001,gamma=10,beta=0.05,w_edge=2,w_line=-5,convergence=0.5)
        
        #hull2vol[i] = coordinates for snake initiation 
        #alpha=snakes energy, tendency to move away from OG initiator (higher alpha=more freedom to move)-small here cus initiated snake close to fascia already 
        #beta= smoothness (higher beta = smoother) w_line = attraction the dark/white (- means attracted to DARK pixels, + to white pixels) w_edge - attraction to edges (higher=more attraction

    return snakevol_p #LIST of coordinates

snakevol_p=primary_snake()

fig, axs = plt.subplots(5, 3, figsize=(18,80))   
for i in range(5):
    axs[i,0].set_title(f"slice {i+1}", fontsize=14)
    axs[i,0].imshow(bilateral_t[i], cmap='bone')

    axs[i,1].imshow(bilateral_t[i], cmap='bone')
    axs[i,1].plot(hull2vol[i][:, 0], hull2vol[i][:, 1], '--r', lw=1)

    axs[i,2].imshow(bilateral_t[i], cmap='bone')
    axs[i,2].plot(snakevol_p[i][:, 0], snakevol_p[i][:, 1], '-b', lw=1)